The purpose of this notebook is to take a subset of the datasets that is big enough to explore and train models on. The datasets right now, are very large and take a long time to load or do any pandas functions on. I will load it in here, and clean it and save the cleaned subset to a csv, so that I don't have to reload the data again.

## Cleaning

In [1]:
import time
notebook_start = time.time()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Insure that all columns and rows are displayed 
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)

In [3]:
start = time.time()
accepted_df = pd.read_csv('Sample_datasets/accepted_2007_to_2018Q4.csv.gz', low_memory=False)
rejected_df = pd.read_csv('Sample_datasets/rejected_2007_to_2018Q4.csv.gz', low_memory=False)
feat_def = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx')
print(accepted_df.shape)
print(f"Reading the csv's take {int((time.time() - start)/60)} minutes.")
accepted_df.head()

(2260701, 151)
Reading the csv's take 2 minutes.


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300

First I will limit the dataset to loans that have charged off or have been fully paid since that is what I'm looking to predict.

In [4]:
accepted_df.loan_status.value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [5]:
# only keep the loans that are paid or charged off.
accepted_df_binary = accepted_df[accepted_df.loan_status.isin(['Fully Paid', 'Charged Off'])].copy()
accepted_df_binary.shape

(1345310, 151)

Just from that we dropped almost a million loans.

In [6]:
pd.options.display.max_rows = None
display(feat_def)
pd.options.display.max_rows = 10

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


A lot of these features are information that is applicable to loans that are already in progress. I won't use those for the model since we want to predict the outcome before the loan was given. There are also features that are not relevant like "id" or "title",  or superfluous like "addr_state" since we have the zipcode. I will drop those too. 

In [7]:
# Go through the features and their descriptions and choose which columns need to be dropped
cols_to_drop = ['addr_state','desc','earliest_cr_line','emp_title','funded_amnt','funded_amnt_inv','id',
                'initial_list_status','last_credit_pull_d','last_fico_range_high','last_fico_range_low',
                'last_pymnt_amnt','last_pymnt_d','member_id','next_pymnt_d','out_prncp','out_prncp_inv','policy_code',
                'purpose','pymnt_plan','sub_grade','title','total_pymnt','total_pymnt_inv','total_rec_int',
                'total_rec_late_fee','total_rec_prncp','url','hardship_flag','hardship_type','hardship_reason',
                'hardship_status','deferral_term','hardship_amount','hardship_start_date','hardship_end_date',
                'payment_plan_start_date','hardship_length','hardship_dpd','hardship_loan_status',
                'orig_projected_additional_accrued_interest','hardship_payoff_balance_amount',
                'hardship_last_payment_amount','disbursement_method','debt_settlement_flag','debt_settlement_flag_date',
                'settlement_status','settlement_date','settlement_amount','settlement_percentage','settlement_term']

In [8]:
# drop_columns
accepted_df_binary.drop(cols_to_drop, axis=1, inplace=True)
print(f'Dropped {len(cols_to_drop)} columns')
accepted_df_binary.shape

Dropped 51 columns


(1345310, 100)

Now that we've narrowed down the columns, we need a sample that's big enough to train a model on, but small enough so that exploring the data and running the models can be done in a timely fashion. If I am not able get decent results from the models, I'll see about increasing the sample size.

Instead of taking a randomized sample, I will take the last three years of loans from the dataset.

In [9]:
# change the issue_d to a datetime object 
accepted_df_binary['issue_d'] = pd.to_datetime(accepted_df_binary['issue_d'])
# take the loans since 2016
last_3_yr = accepted_df_binary[accepted_df_binary.issue_d >= '2016-01'].copy()
# sort the dataframe by date and change the name to date
last_3_yr.sort_values('issue_d', inplace=True)
last_3_yr.reset_index(drop=True, inplace=True)
last_3_yr.rename(columns={'issue_d': 'date'}, inplace=True)

last_3_yr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518706 entries, 0 to 518705
Data columns (total 100 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   loan_amnt                            518706 non-null  float64       
 1   term                                 518706 non-null  object        
 2   int_rate                             518706 non-null  float64       
 3   installment                          518706 non-null  float64       
 4   grade                                518706 non-null  object        
 5   emp_length                           482772 non-null  object        
 6   home_ownership                       518706 non-null  object        
 7   annual_inc                           518706 non-null  float64       
 8   verification_status                  518706 non-null  object        
 9   date                                 518706 non-null  datetime64[ns]


The next step is to get rid of any column that has too many NaN's to use. 

In [10]:
# get a list of the column names that have more than 30% missing values, and drop those columns
thirty_nan = [i for i in last_3_yr.columns 
              if last_3_yr[i].isnull().sum() > 0.3*last_3_yr.shape[0]]
thirty_nan

['mths_since_last_delinq',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'revol_bal_joint',
 'sec_app_fico_range_low',
 'sec_app_fico_range_high',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_mths_since_last_major_derog']

The features about the secondary have a lot of missing values which makes sense.

In [11]:
print('Dropping ', len(thirty_nan), ' columns')
last_3_yr.drop(thirty_nan, axis=1, inplace=True)
last_3_yr.shape

Dropping  21  columns


(518706, 79)

Now we can deal with the rest of the NaN's.

In [12]:
pd.options.display.max_rows = None
display(last_3_yr.isna().sum())
pd.options.display.max_rows = 10

loan_amnt                         0
term                              0
int_rate                          0
installment                       0
grade                             0
emp_length                    35934
home_ownership                    0
annual_inc                        0
verification_status               0
date                              0
loan_status                       0
zip_code                          1
dti                             372
delinq_2yrs                       0
fico_range_low                    0
fico_range_high                   0
inq_last_6mths                    1
open_acc                          0
pub_rec                           0
revol_bal                         0
revol_util                      413
total_acc                         0
recoveries                        0
collection_recovery_fee           0
collections_12_mths_ex_med        0
application_type                  0
acc_now_delinq                    0
tot_coll_amt                

All of the columns wit NaN's in the thousands seem like pretty insignificant features, besides emp_length, I'll drop those columns and afterward drop the rest of the rows with NaN's since they're so few. And again if I can't get good results with the models, I'll come back to this.

In [13]:
# list of columns to drop
cols_to_drop = ['mths_since_rcnt_il', 'il_util', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 
                'mths_since_recent_bc', 'mths_since_recent_inq', 'num_tl_120dpd_2m', 'percent_bc_gt_75',]
last_3_yr.drop(cols_to_drop, axis=1, inplace=True)
last_3_yr.dropna(inplace=True)
print(last_3_yr.shape)
print(f'There are now {last_3_yr.isna().sum().sum()} missing values')

(482335, 70)
There are now 0 missing values


In [14]:
last_3_yr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482335 entries, 0 to 518705
Data columns (total 70 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   loan_amnt                   482335 non-null  float64       
 1   term                        482335 non-null  object        
 2   int_rate                    482335 non-null  float64       
 3   installment                 482335 non-null  float64       
 4   grade                       482335 non-null  object        
 5   emp_length                  482335 non-null  object        
 6   home_ownership              482335 non-null  object        
 7   annual_inc                  482335 non-null  float64       
 8   verification_status         482335 non-null  object        
 9   date                        482335 non-null  datetime64[ns]
 10  loan_status                 482335 non-null  object        
 11  zip_code                    482335 non-

## Sampling

The dataset is still too big to upload to github, but pandas will be able to manage it fine. I'll save this dataframe to a csv to explore and train models, and I'll also take a small sample to uplaod to github.

In [15]:
last_3_yr.reset_index(inplace=True, drop=True)
last_3_yr.to_csv('../accepted_3_yrs.csv', index=False)
accepted_sample = last_3_yr.sample(5000, random_state=0)
accepted_sample.to_csv('data/accepted_sample.csv', index=False)

### Rejected loans

Now I'll do the same process for the rejected loans.

In [16]:
rejected_df

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0
...,...,...,...,...,...,...,...,...,...
27648736,10000.0,2016-12-31,Debt consolidation,590.0,41.26%,441xx,OH,< 1 year,0.0
27648737,10000.0,2016-12-31,moving,NaN,1.48%,207xx,MD,5 years,0.0
27648738,1200.0,2016-12-31,Other,686.0,10.26%,914xx,CA,< 1 year,0.0
27648739,25000.0,2016-12-31,debt_consolidation,NaN,17.71%,880xx,NM,< 1 year,0.0


In [17]:
rejected_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648741 entries, 0 to 27648740
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Application Date      object 
 2   Loan Title            object 
 3   Risk_Score            float64
 4   Debt-To-Income Ratio  object 
 5   Zip Code              object 
 6   State                 object 
 7   Employment Length     object 
 8   Policy Code           float64
dtypes: float64(3), object(6)
memory usage: 1.9+ GB


In [18]:
# take the last 3 years
rejected_df['date'] = pd.to_datetime(rejected_df['Application Date'])
last_3_yr_rejected = rejected_df[rejected_df.date >= '2016-01'].sort_values('date').reset_index(drop=True)
last_3_yr_rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21339229 entries, 0 to 21339228
Data columns (total 10 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Amount Requested      float64       
 1   Application Date      object        
 2   Loan Title            object        
 3   Risk_Score            float64       
 4   Debt-To-Income Ratio  object        
 5   Zip Code              object        
 6   State                 object        
 7   Employment Length     object        
 8   Policy Code           float64       
 9   date                  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 1.6+ GB


There are 21 million rejected loans in these years, and the dataframe is 1.6 GB. This is still too big, and I'm going to have to take a sample for myself here as well. I'll take 2 million rejected loans which is four times the number of accepted loans that there were in those years.

In [19]:
rejected_sample = last_3_yr_rejected.sample(2000000, replace=True, random_state=0)

In [20]:
rejected_sample

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code,date
8325804,15000.0,2017-07-24,debt_consolidation,636.0,23.39%,945xx,CA,5 years,0.0,2017-07-24
18261621,22000.0,2018-09-12,Debt consolidation,NaN,14.98%,805xx,CO,< 1 year,0.0,2018-09-12
2215104,1000.0,2016-07-08,other,NaN,31.48%,856xx,AZ,< 1 year,0.0,2016-07-08
5157699,2000.0,2017-01-25,moving,NaN,12.15%,565xx,MN,5 years,0.0,2017-01-25
15039847,10000.0,2018-05-20,Business Loan,NaN,-1%,394xx,MS,< 1 year,0.0,2018-05-20
...,...,...,...,...,...,...,...,...,...,...
7325348,30000.0,2017-06-06,home_improvement,NaN,20.22%,149xx,NY,< 1 year,0.0,2017-06-06
736236,5000.0,2016-03-07,debt_consolidation,NaN,-1%,396xx,MS,< 1 year,0.0,2016-03-07
16724719,10500.0,2018-07-21,Debt consolidation,NaN,36.45%,890xx,NV,< 1 year,0.0,2018-07-21
12239746,5000.0,2018-01-19,Car financing,NaN,0%,782xx,TX,< 1 year,0.0,2018-01-19


I'm going to use this dataset to compare it with the accepted loans, so I will drop the features that aren't in the accepted loans, and change the columns names so that they match.

In [21]:
# rename columns to match the accepted dataframe and drop the columns that aren't in the accepted loans
rename_dict = {'Application Date': 'date', 'Amount Requested': 'loan_amnt', 
               'Debt-To-Income Ratio': 'dti', 'Employment Length': 'emp_length',
              'Zip Code': 'zip_code'}
rejected_sample.rename(columns=rename_dict, inplace=True)
rejected_sample.drop(['Risk_Score', 'Policy Code', 'State', 'Loan Title'], axis=1, inplace=True)
rejected_sample.dropna(inplace=True)

In [22]:
rejected_sample

,loan_amnt,date,dti,zip_code,emp_length,date
8325804,15000.0,2017-07-24,23.39%,945xx,5 years,2017-07-24
18261621,22000.0,2018-09-12,14.98%,805xx,< 1 year,2018-09-12
2215104,1000.0,2016-07-08,31.48%,856xx,< 1 year,2016-07-08
5157699,2000.0,2017-01-25,12.15%,565xx,5 years,2017-01-25
15039847,10000.0,2018-05-20,-1%,394xx,< 1 year,2018-05-20
...,...,...,...,...,...,...
7325348,30000.0,2017-06-06,20.22%,149xx,< 1 year,2017-06-06
736236,5000.0,2016-03-07,-1%,396xx,< 1 year,2016-03-07
16724719,10500.0,2018-07-21,36.45%,890xx,< 1 year,2018-07-21
12239746,5000.0,2018-01-19,0%,782xx,< 1 year,2018-01-19


This dataframe is still too big to upload to github, but I'll still save this as a csv so I don't need to load the full dataset. and then take a sample of my sample for github.

In [23]:
# save it to a csv
rejected_sample.to_csv('../rejected_3_yrs.csv', index=False)
rejected_sample = rejected_sample.sample(5000, random_state=0)
rejected_sample.to_csv('data/rejected_sample.csv')

In [24]:
notebook_end = time.time()
print(f'This notebook takes {int((time.time() - start)/60)} minutes to run')

This notebook takes 8 minutes to run
